# Construct time series of surface elevation in polygons along centerline

In [ ]:
import xarray as xr
import rioxarray as rxr
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import numpy as np
import os
from shapely.geometry import LineString
from shapely import segmentize
import glob
from tqdm.auto import tqdm

In [ ]:
# -----Base path to data
data_path = '/Users/raineyaberle/Research/PhD/Hubbard/'

# -----DEM file names
h_fns = sorted(glob.glob(os.path.join(data_path, 'surface_elevation', 'surface_elevation_filtered_old', '*.tif')))

# -----Load centerline
cl_path = os.path.join(data_path, 'velocity', 'center.gpkg')
cl = gpd.read_file(cl_path)
# Reproject to Alaska Albers
cl = cl.to_crs('EPSG:3338')
# Increase spatial resolution
def increase_linestring_resolution(line, distance=50):
    # calculate initial distance vector
    x, y = line.coords.xy[0], line.coords.xy[1]
    line_dist = np.zeros(len(line.coords.xy[0]))
    for i in range(1, len(line.coords.xy[0])):
        line_dist[i] = np.sqrt((x[i]-x[i-1])**2 + (y[i]-y[i-1])**2) + line_dist[i-1]
    # create new line distance vector
    new_line_dist = np.arange(0, np.nanmax(line_dist), step=distance)
    # interpolate coordinates on new distance vector
    new_x = np.interp(new_line_dist, line_dist, x)
    new_y = np.interp(new_line_dist, line_dist, y)
    # save as linestring
    new_coords = list(zip(new_x, new_y))
    new_line = LineString(new_coords)
    
    return new_line, new_line_dist
new_geom, cl_dist = increase_linestring_resolution(cl.geometry[0])
cl['geometry'] = [new_geom]

In [ ]:
# -----Check if centerline polygons already exist in file
polygons_fn = os.path.join(data_path, 'velocity', 'centerline_polygons.gpkg')
if os.path.exists(polygons_fn):
    polygons_gdf = gpd.read_file(polygons_fn)
    print('Centerline polygons loaded from file')
else:
    # -----Create polygons from centerline
    cl_x, cl_y = cl.geometry[0].coords.xy
    # Length and width of each polygon
    length, width = 2e3, 2e3
    # Determine distance between split points
    dx = int(length / (cl_dist[1] - cl_dist[0]))
    # Determine number of polygons
    num_polygons = int(np.ceil(len(cl_dist) / dx))
    # Generate the polygons
    polygons = []
    # Iterate over polygons
    for i in range(num_polygons):
        # Crop centerline
        if dx*(i+1) < len(cl_dist):
            Iline = np.arange(dx*i, dx*(i+1))
        else:
            Iline = np.arange(dx*i, len(cl_dist))
        cl_x_line, cl_y_line = [cl_x[i] for i in Iline], [cl_y[i] for i in Iline]
        cl_line = LineString(list(zip(cl_x_line, cl_y_line)))
        # Create a buffer polygon around the LineString 
        polygon = cl_line.buffer(width / 2, cap_style='flat', join_style=2)
        polygons.append(polygon)
    
    # Reformat as geopandas.GeoDataFrame
    polygons_gdf = gpd.GeoDataFrame({'geometry': polygons}, crs=cl.crs)
    # Save to file
    # polygons_gdf.to_file(polygons_fn)
    # print('Centerline polygons saved to file:', polygons_fn)

# Plot results
fig, ax = plt.subplots()
polygons_gdf.plot(ax=ax, facecolor='None', edgecolor='m')
cl.plot(ax=ax, color='k')
plt.show()

## Sample surface elevations in each polygon

In [ ]:
# Iterate over DEM file names
dates = []
datasets = []
h_polys = np.nan*np.zeros((len(h_fns), len(polygons_gdf)))
for i, fn in enumerate(h_fns):
    h_xda = rxr.open_rasterio(fn)
    h_xda = xr.where(np.abs(h_xda) > 10e3, np.nan, h_xda)
    h_xda = h_xda.rio.write_crs('EPSG:3338')
    dates.append(os.path.basename(fn)[0:10])
    datasets.append(os.path.basename(fn).split('_')[1])
    # Sample surface elevations in each polygon
    for j in range(len(polygons_gdf)):
        h_polys[i,j] = np.nanmean(h_xda.rio.clip(polygons_gdf.iloc[j], polygons_gdf.crs).data)
h_polys

In [ ]:
# Reformat as dataframe
h_polys_df = pd.DataFrame({'Date': dates,
                           'Dataset': datasets})
for j in range(len(polygons_gdf)):
    df = pd.DataFrame({f'Polygon {j+1}': h_polys[:,j]})
    h_polys_df = pd.concat([h_polys_df, df], axis=1)
h_polys_df

# Save to file
h_polys_fn = os.path.join(data_path, 'surface_elevation', 'centerline_polygons_surface_elevation.csv')
h_polys_df.to_csv(h_polys_fn, index=False)
print('Centerline polygons surface elevations saved to file:', h_polys_fn)

In [ ]:


h_polys_df['Date'] = pd.DatetimeIndex(h_polys_df['Date'])
h_polys_df['Month'] = pd.DatetimeIndex(h_polys_df['Date']).month
cols = [x for x in h_polys_df.columns if 'Polygon' in x]

# Plot
fig, ax = plt.subplots(len(cols), 1, figsize=(6,len(cols)*4))

for i, col in enumerate(cols):
    ax[i].plot(h_polys_df['Date'], h_polys_df[col], '.b')
plt.show()